In [1]:
import jax
import jax.numpy as jnp
import equinox as eqx
from jax import make_jaxpr

# --- Case A: kappa as a Python float (STATIC) ---
kappa_py = 2.0

@eqx.filter_jit
def f_py(x):
    # kappa_py is captured from the closure.
    return kappa_py * jnp.sum(x**2)

x = jnp.arange(4., dtype=jnp.float32)
print("=== JAXPR with kappa as PY float (static) ===")
print(make_jaxpr(f_py)(x))   # look at invars vs consts


# --- Case B: kappa as a 0-D array (DYNAMIC) ---
kappa_arr = jnp.array(2.0, dtype=jnp.float32)

@eqx.filter_jit
def f_arr(x):
    # kappa_arr is captured but it's an ARRAY leaf → dynamic.
    return kappa_arr * jnp.sum(x**2)

print("\n=== JAXPR with kappa as 0-D ARRAY (dynamic) ===")
print(make_jaxpr(f_arr)(x))


=== JAXPR with kappa as PY float (static) ===
{ lambda ; a:f32[4]. let
    _:i32[] b:f32[] = pjit[
      name=f_py
      jaxpr={ lambda ; a:f32[4]. let
          c:f32[4] = integer_pow[y=2] a
          d:f32[] = reduce_sum[axes=(0,)] c
          b:f32[] = mul 2.0:f32[] d
        in (0:i32[], b) }
    ] a
  in (b,) }

=== JAXPR with kappa as 0-D ARRAY (dynamic) ===
{ lambda ; a:f32[4]. let
    _:i32[] b:f32[] = pjit[
      name=f_arr
      jaxpr={ lambda c:f32[]; a:f32[4]. let
          d:f32[4] = integer_pow[y=2] a
          e:f32[] = reduce_sum[axes=(0,)] d
          b:f32[] = mul c e
        in (0:i32[], b) }
    ] a
  in (b,) }


In [7]:
import jax
from jax import make_jaxpr
import jax.numpy as jnp
import equinox as eqx
from equinox import filter_make_jaxpr

In [9]:
# A PyTree argument with mixed leaves
arg = {
    "W": jnp.ones((2,2), dtype=jnp.float32),  # array → dynamic
    "name": "layer",                           # str   → static
    "use_bias": False,                          # bool  → static
}

@eqx.filter_jit
def forward(mod, x):
    y = mod["W"] @ x
    return y + 1 if mod["use_bias"] else y

x = jnp.ones((2,), dtype=jnp.float32)
print("\n=== JAXPR for mixed PyTree argument (arrays dynamic; others static) ===")
print(filter_make_jaxpr(forward)(arg, x))



=== JAXPR for mixed PyTree argument (arrays dynamic; others static) ===
({ lambda ; a:f32[2,2] b:f32[2]. let
    _:i32[] c:f32[2] = pjit[
      name=forward
      jaxpr={ lambda ; a:f32[2,2] b:f32[2]. let
          c:f32[2] = dot_general[
            dimension_numbers=(([1], [0]), ([], []))
            preferred_element_type=float32
          ] a b
        in (0:i32[], c) }
    ] a b
  in (c,) }, ShapeDtypeStruct(shape=(2,), dtype=float32), None)


In [14]:
from typing import Union

In [16]:
class Tiny(eqx.Module):
    kappa: Union[float, jax.Array]        # dynamic (array leaf)
    scheme: str             # static (python)
    def __init__(self):
        self.kappa = 1.5 #jnp.array(1.5, dtype=jnp.float32)
        self.scheme = "fourier"

    def rhs(self, u, t):
        # both captured through 'self'
        return self.kappa * (u + t)

model = Tiny()

@eqx.filter_jit
def rhs_wrapped(u, t):
    return model.rhs(u, t)

u = jnp.ones((3,), dtype=jnp.float32)
t = jnp.array(0.1, dtype=jnp.float32)

print("\n=== JAXPR class capture: array field dynamic; python field static ===")
print(make_jaxpr(rhs_wrapped)(u, t))



=== JAXPR class capture: array field dynamic; python field static ===
{ lambda ; a:f32[3] b:f32[]. let
    _:i32[] c:f32[3] = pjit[
      name=rhs_wrapped
      jaxpr={ lambda ; a:f32[3] b:f32[]. let
          d:f32[3] = add a b
          c:f32[3] = mul 1.5:f32[] d
        in (0:i32[], c) }
    ] a b
  in (c,) }
